In [ ]:
import itertools
import json
import os
import pickle
import re
import shutil
import subprocess
import sys
from functools import partial

from itertools import cycle
from pathlib import Path
from typing import Any, Dict

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scri
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from spherical_functions import LM_index as lm


base_path = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs")

# Functions

In [ ]:
def get_Lm_info_from_col_name(col_name):
    real = "Re" in col_name
    L = int(col_name.split("(")[1].split(",")[0].strip())
    m = int(col_name.split("(")[1].split(",")[1][:-1].strip())
    return L,m,real

def set_higher_L_modes_to_zero(
    input_file, output_file, min_L_to_keep=1e10
):
    shutil.copy(input_file, output_file)
    with h5py.File(output_file, "r+") as outfile:
        for key, data in outfile.items():
            if "Version" in key:
                continue
            if isinstance(data, h5py.Dataset):
                print(f"----Modifying {key}")
                legend = data.attrs['Legend']
                for i,col_name in enumerate(list(legend)):
                    if col_name == 'time':
                        continue
                    L, m, real = get_Lm_info_from_col_name(col_name)
                    if L > min_L_to_keep:
                        data[:, i] = 0.0


In [ ]:
def add_uniform_noise(
    input_file, output_file, noise_amp=1e-16, add_relative_noise=False, min_t = -10.0
):
    shutil.copy(input_file, output_file)
    with h5py.File(output_file, "r+") as outfile:
        for key, data in outfile.items():
            if "Version" in key:
                continue
            if isinstance(data, h5py.Dataset):
                print(f"----Modifying {key}")
                t = data[:, 0]
                #find the index of the first time greater than min_t
                first_index = np.searchsorted(t, min_t)
                if add_relative_noise:
                    data[first_index:, 1:] = data[first_index:, 1:] * (
                        1
                        + noise_amp
                        * np.random.uniform(low=-1, high=1, size=data[first_index:, 1:].shape)
                    )
                else:
                    data[first_index:, 1:] = data[first_index:, 1:] + noise_amp * (
                        np.random.uniform(low=-1, high=1, size=data[first_index:, 1:].shape)
                    )


def remove_values_below_tolerance(
    input_file, output_file, tolerance=1e-16, use_relative_tolerance=False
):
    shutil.copy(input_file, output_file)
    with h5py.File(output_file, "r+") as outfile:
        for key, data in outfile.items():
            if "Version" in key:
                continue
            if isinstance(data, h5py.Dataset):
                print(f"----Modifying {key}")
                data_no_t = data[:, 1:]

                if use_relative_tolerance:
                    max_per_row = np.max(np.abs(data_no_t), axis=1)
                    thresh = (tolerance * max_per_row)[:, np.newaxis]
                else:
                    # absolute threshold
                    thresh = tolerance
                cleaned = np.where(np.abs(data_no_t) < thresh, 0, data_no_t)
                data[:, 1:] = cleaned

def just_link(input_file, output_file):
    os.symlink(input_file, output_file)

def get_diff_WT_data(file1,file2):
    with h5py.File(file1, "r") as f1, h5py.File(file2, "r") as f2:
        vars = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
        diff_data = {}
        for key in vars:
            print(f"----Taking diff: {key}")
            diff_data[key] = np.array(f2[key])
            diff_data[key] = diff_data[key][:,1:] - f1[key][:,1:]
        return diff_data

In [ ]:

def make_config_file(
    BoundaryDataPath: Path,
    InputSavePath: Path = None,
    which_ID: str = "ConformalFactor",
    options_dict_user={},
    observer_vol_data = False,
) -> Path:
    options_dict = {
        "Cce.Evolution.TimeStepper.AdamsBashforth.Order": 3,
        "Cce.Evolution.StepChoosers.Constant": 0.1,
        "Cce.Evolution.StepChoosers.ErrorControl(SwshVars).AbsoluteTolerance": 1e-9,
        "Cce.Evolution.StepChoosers.ErrorControl(SwshVars).RelativeTolerance": 1e-7,
        "Cce.Evolution.StepChoosers.ErrorControl(CoordVars).AbsoluteTolerance": 1e-9,
        "Cce.Evolution.StepChoosers.ErrorControl(CoordVars).RelativeTolerance": 1e-8,
        "Cce.LMax": 20,
        "Cce.NumberOfRadialPoints": 15,
        "Cce.ObservationLMax": 8,
        "Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MinOrder": 10,
        "Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MaxOrder": 10,
        "Cce.H5LookaheadTimes": 10000,
        "Cce.Filtering.RadialFilterHalfPower": 64,
        "Cce.Filtering.RadialFilterAlpha": 35.0,
        "Cce.Filtering.FilterLMax": 18,
        "Cce.ScriInterpOrder": 5,
        "Cce.ScriOutputDensity": 1,
    }

    for key in options_dict_user.keys():
        if key not in options_dict:
            raise ValueError(f"Key {key} is not a valid option.")
        else:
            options_dict[key] = options_dict_user[key]

    CCE_ID_data = ""
    match which_ID:
        case "ConformalFactor":
            CCE_ID_data = """
    ConformalFactor:
      AngularCoordTolerance: 1e-13
      MaxIterations: 1000 # Do extra iterations in case we improve.
      RequireConvergence: False # Often don't converge to 1e-13, but that's fine
      OptimizeL0Mode: True
      UseBetaIntegralEstimate: False
      ConformalFactorIterationHeuristic: SpinWeight1CoordPerturbation
      UseInputModes: False
      InputModes: []
"""
        case "InverseCubic":
            CCE_ID_data = """
    InverseCubic:
"""
        case "ZeroNonSmooth":
            CCE_ID_data = """
    ZeroNonSmooth:
      AngularCoordTolerance: 1e-13
      MaxIterations: 1000
      RequireConvergence: False
"""
        case "NoIncomingRadiation":
            CCE_ID_data = """
    NoIncomingRadiation:
      AngularCoordTolerance: 1e-13
      MaxIterations: 1000
      RequireConvergence: False
"""
    if InputSavePath is None:
        InputSavePath = BoundaryDataPath.parent / "cce.yaml"
    assert InputSavePath.parent.exists()

    config_file = f"""
# Distributed under the MIT License.
# See LICENSE.txt for details.

# This block is used by testing and the SpECTRE command line interface.
Executable: CharacteristicExtract
Testing:
  Check: parse
  Priority: High

---
Evolution:
  InitialTimeStep: 0.25
  MinimumTimeStep: 1e-7
  InitialSlabSize: 10.0

ResourceInfo:
  AvoidGlobalProc0: false
  Singletons: Auto

Observers:
  VolumeFileName: "vol_{str(InputSavePath.stem)}"
  ReductionFileName: "red_{str(InputSavePath.stem)}"

EventsAndTriggersAtSlabs:
  # Write the CCE time step every Slab. A Slab is a fixed length of simulation
  # time and is not influenced by the dynamically adjusted step size.
  - Trigger:
      Slabs:
        EvenlySpaced:
          Offset: 0
          Interval: 1
    Events:
      - ObserveTimeStep:
          # The output is written into the "ReductionFileName" HDF5 file under
          # "/SubfileName.dat"
          SubfileName: CceTimeStep
          PrintTimeToTerminal: true
    #   - ObserveFields:
    #       VariablesToObserve:
    #         - BondiBeta
    #         - Du(J)
    #         - DuRDividedByR
    #         - Dy(BondiBeta)
    #         - Dy(Du(J))
    #         - Dy(Dy(BondiBeta))
    #         - Dy(Dy(Du(J)))
    #         - Dy(Dy(J))
    #         - Dy(Dy(Q))
    #         - Dy(Dy(U))
    #         - Dy(Dy(W))
    #         - Dy(H)
    #         - Dy(J)
    #         - Dy(Q)
    #         - Dy(U)
    #         - Dy(W)
    #         - EthRDividedByR
    #         - H
    #         - InertialRetardedTime
    #         - J
    #         - OneMinusY
    #         - Psi0
    #         - Psi1
    #         - Q
    #         - R
    #         - U
    #         - W

EventsAndTriggersAtSteps:

Cce:
  Evolution:
    TimeStepper:
      AdamsBashforth:
        Order: {options_dict['Cce.Evolution.TimeStepper.AdamsBashforth.Order']} # Going to higher order doesn't seem necessary for CCE
    StepChoosers:
      - Constant: {options_dict['Cce.Evolution.StepChoosers.Constant']} # Don't take steps bigger than 0.1M
      - LimitIncrease:
          Factor: 2
      - ErrorControl(SwshVars):
          AbsoluteTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(SwshVars).AbsoluteTolerance']}
          RelativeTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(SwshVars).RelativeTolerance']}
          # These factors control how much the time step is changed at once.
          MaxFactor: 2
          MinFactor: 0.25
          # How close to the "perfect" time step we take. Since the "perfect"
          # value assumes a linear system, we need some safety factor since our
          # system is nonlinear, and also so that we reduce how often we retake
          # time steps.
          SafetyFactor: 0.9
      - ErrorControl(CoordVars):
          AbsoluteTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(CoordVars).AbsoluteTolerance']}
          RelativeTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(CoordVars).RelativeTolerance']}
          # These factors control how much the time step is changed at once.
          MaxFactor: 2
          MinFactor: 0.25
          # How close to the "perfect" time step we take. Since the "perfect"
          # value assumes a linear system, we need some safety factor since our
          # system is nonlinear, and also so that we reduce how often we retake
          # time steps.
          SafetyFactor: 0.9

  # The number of angular modes used by the CCE evolution. This must be larger
  # than ObservationLMax. We always use all of the m modes for the LMax since
  # using fewer m modes causes aliasing-driven instabilities.
  LMax: {options_dict['Cce.LMax']}
  # Probably don't need more than 15 radial grid points, but could increase
  # up to ~20
  NumberOfRadialPoints: {options_dict['Cce.NumberOfRadialPoints']}
  # The maximum ell we use for writing waveform output. While CCE can dump
  # more, you should be cautious with higher modes since mode mixing, truncation
  # error, and systematic numerical effects can have significant contamination
  # in these modes.
  ObservationLMax: {options_dict['Cce.ObservationLMax']}

  InitializeJ:
    # To see what other J-initialization procedures are available, comment
    # out this group of options and do, e.g. "Blah:" The code will print
    # an error message with the available options and a help string.
    # More details can be found at spectre-code.org.
{CCE_ID_data}

  StartTime: Auto
  EndTime: Auto
  ExtractionRadius: Auto

  BoundaryDataFilename: {BoundaryDataPath.name}
  H5Interpolator:
    BarycentricRationalSpanInterpolator:
      MinOrder: {options_dict['Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MinOrder']}
      MaxOrder: {options_dict['Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MaxOrder']}

  H5LookaheadTimes: {options_dict['Cce.H5LookaheadTimes']}

  Filtering:
    RadialFilterHalfPower: {options_dict['Cce.Filtering.RadialFilterHalfPower']}
    RadialFilterAlpha: {options_dict['Cce.Filtering.RadialFilterAlpha']}
    FilterLMax: {options_dict['Cce.Filtering.FilterLMax']}

  ScriInterpOrder: {options_dict['Cce.ScriInterpOrder']}
  ScriOutputDensity: {options_dict['Cce.ScriOutputDensity']}

"""

    with InputSavePath.open("w") as f:
        f.writelines(config_file)

    return InputSavePath


def make_submit_file(
    save_folder_path: Path,
    cce_input_file_path: Path,
    CCE_Executable_path: Path,
    write_scripts_only=False,
):
    submit_script = f"""#!/bin/bash -
#SBATCH -J CCE_{save_folder_path.stem}             # Job Name
#SBATCH -o CCE.stdout                 # Output file name
#SBATCH -e CCE.stderr                 # Error file name
#SBATCH -n 2                          # Number of cores
#SBATCH -p expansion                  # Queue name
#SBATCH --ntasks-per-node 2           # number of MPI ranks per node
#SBATCH -t 24:0:00   # Run time
#SBATCH -A sxs                # Account name
#SBATCH --no-requeue
#SBATCH --reservation=sxs_standing

# Spectre related stuff to load JeMalloc
export SPECTRE_HOME=/central/groups/sxs/hchaudha/spectre
. $SPECTRE_HOME/support/Environments/caltech_hpc_gcc.sh
spectre_load_modules

# Go to the correct folder with the boundary data
cd {save_folder_path}

# run CCE
{CCE_Executable_path} --input-file ./{cce_input_file_path.name}
"""
    submit_script_path = save_folder_path / "submit.sh"
    submit_script_path.write_text(submit_script)

    if not write_scripts_only:
        command = f"cd {save_folder_path} && qsub {submit_script_path}"
        status = subprocess.run(command, capture_output=True, shell=True, text=True)
        if status.returncode == 0:
            print(f"Succesfully submitted {submit_script_path}\n{status.stdout}")
        else:
            sys.exit(
                f"Job submission failed for {submit_script_path} with error: \n{status.stdout} \n{status.stderr}"
            )


def create_CCE_folder(
    NewCCEPath: Path,
    BondiBaseH5File: Path,
    CCE_Executable_path: Path,
    WT_data_modification_function,
    CCE_ID: str = "ConformalFactor",
    options_dict_user={},
    write_scripts_only=False,

):
    NewCCEPath = NewCCEPath.resolve()
    BondiBaseH5File = BondiBaseH5File.resolve()
    CCE_Executable_path = CCE_Executable_path.resolve()

    if not BondiBaseH5File.exists():
        raise Exception(f"{BondiBaseH5File} does not exist!")
    if not CCE_Executable_path.exists():
        raise Exception(f"{CCE_Executable_path} does not exist!")

    NewCCEPath.mkdir(parents=True, exist_ok=False)

    WT_data_modification_function(BondiBaseH5File, NewCCEPath/BondiBaseH5File.name)

    make_config_file(
        BoundaryDataPath=NewCCEPath / BondiBaseH5File.name,
        InputSavePath=NewCCEPath / "cce.yaml",
        which_ID=CCE_ID,
        options_dict_user=options_dict_user,
    )

    make_submit_file(
        save_folder_path=NewCCEPath,
        cce_input_file_path=NewCCEPath / "cce.yaml",
        CCE_Executable_path=CCE_Executable_path,
        write_scripts_only=write_scripts_only,
    )

    print("DONE!\n\n\n\n\n")



In [ ]:
# create_CCE_folder(
#     NewCCEPath=Path(
#         "/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/original_tol_10"
#     ),
#     BondiBaseH5File=Path(
#         "/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/data/BondiCceR0250.h5"
#     ),
#     CCE_Executable_path=Path(
#         "/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/data/CharacteristicExtract"
#     ),
#     WT_data_modification_function=partial(
#         add_uniform_noise, noise_amp=tol, add_relative_noise=True
#     ),
#     CCE_ID="InverseCubic",
#     options_dict_user={
#         "Cce.Evolution.TimeStepper.AdamsBashforth.Order": 3,
#         "Cce.Evolution.StepChoosers.Constant": 0.1,
#         "Cce.Evolution.StepChoosers.ErrorControl(SwshVars).AbsoluteTolerance": 1e-9*10,
#         "Cce.Evolution.StepChoosers.ErrorControl(SwshVars).RelativeTolerance": 1e-7*10,
#         "Cce.Evolution.StepChoosers.ErrorControl(CoordVars).AbsoluteTolerance": 1e-9*10,
#         "Cce.Evolution.StepChoosers.ErrorControl(CoordVars).RelativeTolerance": 1e-8*10,
#         "Cce.LMax": 20,
#         "Cce.NumberOfRadialPoints": 15,
#         "Cce.ObservationLMax": 8,
#         "Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MinOrder": 10,
#         "Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MaxOrder": 10,
#         "Cce.H5LookaheadTimes": 10000,
#         "Cce.Filtering.RadialFilterHalfPower": 64,
#         "Cce.Filtering.RadialFilterAlpha": 35.0,
#         "Cce.Filtering.FilterLMax": 18,
#         "Cce.ScriInterpOrder": 5,
#         "Cce.ScriOutputDensity": 1,
#     }
# )


# Tests

In [ ]:
bondi_data_path = Path("/groups/sxs/hchaudha/scripts/CCE_read_and_modify_WT_data/del/original_data/BondiCceR0250.h5")
save_base_path = Path("/groups/sxs/hchaudha/scripts/CCE_read_and_modify_WT_data/del/modified_data")

In [ ]:
add_uniform_noise(bondi_data_path, save_base_path / "BondiCceR0250_noisy.h5", noise_amp=1e-15, add_relative_noise=True)
diff_dict = get_diff_WT_data(bondi_data_path, save_base_path / "BondiCceR0250_noisy.h5")

In [ ]:
remove_values_below_tolerance(
    bondi_data_path,
    save_base_path / "BondiCceR0250_filtered.h5",
    tolerance=1e-12,
    use_relative_tolerance=True,
)
diff_dict = get_diff_WT_data(
    bondi_data_path, save_base_path / "BondiCceR0250_filtered.h5"
)


In [ ]:
for key, value in diff_dict.items():
    print(f"{key}: {np.max(value[:,1:])}")

In [ ]:
# with h5py.File(bondi_data_path, "r") as f:
with h5py.File(save_base_path / "BondiCceR0250_filtered.h5", "r") as f:
    vars = [
        "Beta.dat",
        "DrJ.dat",
        "DuR.dat",
        "H.dat",
        "J.dat",
        "Q.dat",
        "R.dat",
        "U.dat",
        "W.dat",
    ]
    L2_vals = {}
    print(f.keys())
    for key in vars:
        print(f"{key}: {f[key].shape}")
        L2_vals[key] = np.max(f[key][2000:,1:],axis=1)
        # L2_vals[key] = np.array(f[key][2000:, 1:])
        # L2_vals[key] = np.linalg.norm(f[key][2000:,1:],axis=1)


In [ ]:
for key, value in L2_vals.items():
    if key != 'R.dat':
        continue
    # if key != 'H.dat':
    #     continue
    print(f"{key}: {value.shape}")
    plt.plot(value, label=key)
plt.xlabel('Time')
plt.yscale('log')
plt.ylabel('L2 Norm')
plt.legend()

In [ ]:
plt.plot(np.abs(L2_vals['J.dat'][1000,:]), label='R.dat')
plt.yscale('log')

In [ ]:
base_path = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs")
red_file1_path = base_path / "uni_noise_abs_1e-22/red_cce.h5"
red_file2_path = base_path / "uni_noise_abs_1e-14/red_cce.h5"

red_file1_path = base_path / "uni_noise_rel_1e-16/red_cce.h5"
red_file2_path = base_path / "uni_noise_rel_1e-08/red_cce.h5"
red_file2_path = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/uni_noise_abs_1e-22/red_cce.h5")


red_file1_path = base_path / "filter_rel_1e-16/red_cce.h5"
# red_file2_path = base_path / "filter_rel_1e-08/red_cce.h5"

In [ ]:
with h5py.File(red_file1_path, "r") as f1, h5py.File(red_file2_path, "r") as f2:
    red_vars = ['News', 'Psi0', 'Psi1', 'Psi2', 'Psi3', 'Psi4', 'Strain']
    val1 = f1['SpectreR0250.cce'][red_vars[0]][()]
    val2 = f2['SpectreR0250.cce'][red_vars[0]][()]
    t = val1[:, 0]
    l2_diff = np.linalg.norm(val1[:, 1:] - val2[:, 1:], axis=1)


In [ ]:

plt.plot(t,l2_diff, label='Diff')
plt.yscale('log')

## Use L to change things

In [ ]:
bondi_data_path = Path("/groups/sxs/hchaudha/scripts/CCE_read_and_modify_WT_data/del/original_data/BondiCceR0250.h5")
save_base_path = Path("/groups/sxs/hchaudha/scripts/CCE_read_and_modify_WT_data/del/modified_data")

In [ ]:
set_higher_L_modes_to_zero(
    bondi_data_path, save_base_path/"L10_zero.h5", min_L_to_keep=10
)

In [ ]:

with h5py.File(save_base_path/"L10_zero.h5", "r") as f:
    vars = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
    vars = ['Beta.dat']
    for key in vars:
        legend = f[key].attrs['Legend']
        for i,col_name in enumerate(list(legend)):
            if col_name == 'time':
                continue
            L, m, real = get_Lm_info_from_col_name(col_name)
            print(f"{col_name} : {np.linalg.norm(f[key][:, i])}")
        #     print(f"{col_name}: {real}, {L}, {m}")

# Comparison plots

In [ ]:
base_path = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs")
save_fig_plot = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/figures")

In [ ]:
base_run = base_path / "original/red_cce.h5"
l2_diff_dict = {
    "1e-08": {},
    "1e-10": {},
    "1e-12": {},
    "1e-16": {},
    "1e-18": {},
    "1e-20": {},
    "1e-22": {},
}
red_vars = ['News', 'Psi0', 'Psi1', 'Psi2', 'Psi3', 'Psi4', 'Strain']
with h5py.File(base_run, "r") as f:
    for key in l2_diff_dict.keys():
        run2 = base_path / f"uni_noise_abs_{key}/red_cce.h5"
        with h5py.File(run2, "r") as f2:
            for var in red_vars:
                val1 = f['SpectreR0250.cce'][var]
                val2 = f2['SpectreR0250.cce'][var]
                l2_diff = np.linalg.norm(val1[:, 1:] - val2[:, 1:], axis=1)
                l2_diff_dict[key][var] = l2_diff
                print(f"{key} {var}")
    t = f['SpectreR0250.cce'][red_vars[0]][:, 0]

invert_l2_diff_dict = {}
for key, value in l2_diff_dict.items():
    for var, l2_diff in value.items():
        if var not in invert_l2_diff_dict:
            invert_l2_diff_dict[var] = {}
        invert_l2_diff_dict[var][key] = l2_diff

In [ ]:
for var in red_vars:
    for run, value in invert_l2_diff_dict[var].items():
        plt.plot(t, value, label=run)
    plt.yscale('log')
    plt.xlabel('Time')
    plt.ylabel('L2 Diff Norm')
    plt.legend()
    plt.title(f'uni_noise_abs: Diff {var}')
    plt.yscale('log')
    plt.savefig(save_fig_plot / f"uni_noise_abs_diff_{var}.png")
    plt.clf()


In [ ]:
base_run = base_path / "original/red_cce.h5"
l2_diff_dict = {
    "1e-05": {},
    "1e-06": {},
    "1e-07": {},
    "1e-08": {},
    "1e-10": {},
    "1e-12": {},
    "1e-14": {},
    "1e-16": {},
}
red_vars = ['News', 'Psi0', 'Psi1', 'Psi2', 'Psi3', 'Psi4', 'Strain']
red_vars = ['News']
with h5py.File(base_run, "r") as f:
    for key in l2_diff_dict.keys():
        run2 = base_path / f"uni_noise_rel_{key}/red_cce.h5"
        with h5py.File(run2, "r") as f2:
            for var in red_vars:
                val1 = f['SpectreR0250.cce'][var]
                val2 = f2['SpectreR0250.cce'][var]
                l2_diff = np.linalg.norm(val1[:, 1:] - val2[:, 1:], axis=1)
                l2_diff_dict[key][var] = l2_diff
                print(f"{key} {var}")
    t = f['SpectreR0250.cce'][red_vars[0]][:, 0]

invert_l2_diff_dict = {}
for key, value in l2_diff_dict.items():
    for var, l2_diff in value.items():
        if var not in invert_l2_diff_dict:
            invert_l2_diff_dict[var] = {}
        invert_l2_diff_dict[var][key] = l2_diff

In [ ]:
for var in red_vars:
    for run, value in invert_l2_diff_dict[var].items():
        plt.plot(t, value, label=run)
    plt.yscale('log')
    plt.xlabel('Time')
    plt.ylabel('L2 Diff Norm')
    plt.legend()
    plt.title(f'uni_noise_rel: Diff {var}')
    plt.yscale('log')
    plt.savefig(save_fig_plot / f"uni_noise_rel_diff_{var}.png")
    plt.clf()


In [ ]:
base_run = base_path / "original/red_cce.h5"
l2_diff_dict = {
    "1e-05": {},
    "1e-06": {},
    "1e-07": {},
    "1e-08": {},
    "1e-10": {},
    "1e-12": {},
    "1e-14": {},
    "1e-16": {},
}
red_vars = ['News', 'Psi0', 'Psi1', 'Psi2', 'Psi3', 'Psi4', 'Strain']
red_vars = ['News', 'Strain']
with h5py.File(base_run, "r") as f:
    for key in l2_diff_dict.keys():
        run2 = base_path / f"filter_rel_{key}/red_cce.h5"
        with h5py.File(run2, "r") as f2:
            for var in red_vars:
                val1 = f['SpectreR0250.cce'][var][()]
                val2 = f2['SpectreR0250.cce'][var][()]
                
                t1, data1 = val1[:, 0], val1[:, 1:]
                t2, data2 = val2[:, 0], val2[:, 1:]

                # Define common time grid using intersection or union
                common_time = np.union1d(t1, t2)

                # Interpolate both data sets to the common time grid
                interp1 = interp1d(t1, data1, axis=0, bounds_error=False, fill_value="extrapolate",kind='cubic')
                interp2 = interp1d(t2, data2, axis=0, bounds_error=False, fill_value="extrapolate",kind='cubic')

                aligned1 = interp1(common_time)
                aligned2 = interp2(common_time)

                # Compute L2 norm of the difference across angular modes
                l2_diff = np.linalg.norm(aligned1 - aligned2, axis=1)
                l2_diff_dict[key][var] = (common_time,l2_diff)
                print(f"{key} {var}")

invert_l2_diff_dict = {}
for key, value in l2_diff_dict.items():
    for var, l2_diff in value.items():
        if var not in invert_l2_diff_dict:
            invert_l2_diff_dict[var] = {}
        invert_l2_diff_dict[var][key] = l2_diff

In [ ]:
for var in red_vars:
    for run, value in invert_l2_diff_dict[var].items():
        common_time, value = value
        plt.plot(common_time, value, label=run)
    plt.yscale('log')
    plt.xlabel('Time')
    plt.ylabel('L2 Diff Norm')
    plt.legend()
    plt.title(f'filter_rel(cubic): Diff {var}')
    plt.yscale('log')
    plt.savefig(save_fig_plot / f"filter_rel(cubic)_{var}.png")
    plt.clf()

In [ ]:
for var in red_vars:
    for run, value in invert_l2_diff_dict[var].items():
        plt.plot(t, value, label=run)
    plt.yscale('log')
    plt.xlabel('Time')
    plt.ylabel('L2 Diff Norm')
    plt.legend()
    plt.title(f'filter_rel: Diff {var}')
    plt.yscale('log')
    plt.savefig(save_fig_plot / f"filter_rel_diff_{var}.png")
    plt.clf()

## Original comparison

#### Ode tol

In [ ]:
save_fig_folder_path = save_fig_plot/'diff_base'
if not save_fig_folder_path.exists():
    save_fig_folder_path.mkdir(parents=True, exist_ok=True)

In [ ]:
base_run = base_path / "original/red_cce.h5"
base_run = base_path / "original_tol_0.001/red_cce.h5"
l2_diff_dict_ode = {
    "original_tol_1000": {},
    "original_tol_500": {},
    "original_tol_100": {},
    "original_tol_50": {},
    "original_tol_20": {},
    "original_tol_10": {},
    "original_tol_5": {},
    "original": {},
    "original_tol_1.1": {},
    "original_tol_010": {},
    "original_tol_0.2": {},
    "original_tol_0.02": {},
    "original_tol_0.01": {},
    # "original_tol_0.001": {},
}
base_run = base_path / "rad_20_11factor_tol_0.2/red_cce.h5"
l2_diff_dict_ode = {
    "rad_20_11factor_tol_1000": {},
    "rad_20_11factor_tol_500": {},
    "rad_20_11factor_tol_100": {},
    "rad_20_11factor_tol_50": {},
    "rad_20_11factor_tol_20": {},
    "rad_20_11factor_tol_5": {},
    "rad_20_11factor_tol_1.1": {},
    "rad_20_11factor_tol_1": {},
    "rad_20_11factor_tol_0.2": {},
    # "rad_20_11factor_tol_0.02": {},
    # "rad_20_11factor_tol_0.01": {},
    # "rad_20_11factor_tol_0.001": {},
}

base_run = base_path / "1rad30_ode_tol/r30_tol_0.01/red_cce.h5"
l2_diff_dict_ode = {
    "1rad30_ode_tol/r30_tol_0.1": {},
    # "1rad30_ode_tol/r30_tol_0.01":{},
    # "1rad30_ode_tol/r30_tol_0.001":{},
    "1rad30_ode_tol/r30_tol_1.1": {},
    "1rad30_ode_tol/r30_tol_10": {},
    "1rad30_ode_tol/r30_tol_100": {},
    "1rad30_ode_tol/r30_tol_1000": {},
}
base_run = base_path / "1rad30_ode_tol/r30_start_2000_tol_1.1/red_cce.h5"
l2_diff_dict_ode = {
"1rad30_ode_tol/r30_start_2000_tol_0.1":{},
"1rad30_ode_tol/r30_start_2000_tol_0.01":{},
"1rad30_ode_tol/r30_start_2000_tol_0.001":{},
# "1rad30_ode_tol/r30_start_2000_tol_1.1":{},
"1rad30_ode_tol/r30_start_2000_tol_10":{},
"1rad30_ode_tol/r30_start_2000_tol_100":{},
"1rad30_ode_tol/r30_start_2000_tol_1000":{},
}

# r = 100
# base_run = base_path / f"different_ex_rad_R_convg/{r}_38/red_cce.h5"
# l2_diff_dict_ode = {
# f"different_ex_rad_R_convg/{r}_24":{},
# f"different_ex_rad_R_convg/{r}_28":{},
# f"different_ex_rad_R_convg/{r}_30":{},
# f"different_ex_rad_R_convg/{r}_34":{},
# f"different_ex_rad_R_convg/{r}_38":{},
# }

r=500
base_run = base_path / f"36_segs_R_conv/{r}_38/red_cce.h5"
l2_diff_dict_ode = {
f"36_segs_R_conv/{r}_24":{},
f"36_segs_R_conv/{r}_28":{},
f"36_segs_R_conv/{r}_30":{},
f"36_segs_R_conv/{r}_34":{},
# f"36_segs_R_conv/{r}_38":{},
}

r=500
base_run = base_path / f"36_segs_R_conv/{r}_R_30_L_24/red_cce.h5"
l2_diff_dict_ode = {
f"36_segs_R_conv/{r}_R_30_L_14":{},
f"36_segs_R_conv/{r}_R_30_L_16":{},
f"36_segs_R_conv/{r}_R_30_L_18":{},
f"36_segs_R_conv/{r}_R_30_L_20":{},
f"36_segs_R_conv/{r}_R_30_L_22":{},
}

r = 250
base_run = base_path / "max_step_size_test/max_step_0.04/red_cce.h5"
# base_run = base_path / "max_step_size_test/max_step_0.01/red_cce.h5"
l2_diff_dict_ode = {
# "max_step_0.04":{},
"max_step_size_test/max_step_0.08":{},
"max_step_size_test/max_step_0.07":{},
"max_step_size_test/max_step_0.06":{},
"max_step_size_test/max_step_0.05":{},
# "max_step_size_test/max_step_0.04":{},
"max_step_size_test/max_step_0.03":{},
"max_step_size_test/max_step_0.02":{},
"max_step_size_test/max_step_0.01":{},
# "max_step_size_test/max_step_0.03":{},
}

red_vars = ["News", "Psi0", "Psi1", "Psi2", "Psi3", "Psi4", "Strain"]
# red_vars = ["Psi0", "Psi1"]


with h5py.File(base_run, "r") as f:
    for key in l2_diff_dict_ode.keys():
        run2 = base_path / f"{key}/red_cce.h5"
        with h5py.File(run2, "r") as f2:
            for var in red_vars:
                val1 = f[f"SpectreR0{r}.cce"][var][()]
                val2 = f2[f"SpectreR0{r}.cce"][var][()]

                t1, data1 = val1[:, 0], val1[:, 1:]
                t2, data2 = val2[:, 0], val2[:, 1:]

                # Define common time grid using intersection or union
                max_time = np.min([np.max(t1), np.max(t2)])
                min_time = np.max([np.min(t1), np.min(t2)])
                # get time grid where delta t is 1
                common_time = np.linspace(
                    min_time,
                    max_time,
                    num=int(max_time - min_time),
                )

                # Interpolate both data sets to the common time grid
                interp1 = interp1d(
                    t1,
                    data1,
                    axis=0,
                    bounds_error=False,
                    fill_value="extrapolate",
                    kind="cubic",
                )
                interp2 = interp1d(
                    t2,
                    data2,
                    axis=0,
                    bounds_error=False,
                    fill_value="extrapolate",
                    kind="cubic",
                )

                aligned1 = interp1(common_time)
                aligned2 = interp2(common_time)

                # Compute L2 norm of the difference across angular modes
                l2_diff = np.linalg.norm(aligned1 - aligned2, axis=1)
                l2_diff_dict_ode[key][var] = (common_time, l2_diff)
                print(f"{key} {var}")

                # break

    t = f[f"SpectreR0{r}.cce"][red_vars[0]][:, 0]


invert_l2_diff_dict_ode = {}
for key, value in l2_diff_dict_ode.items():
    for var, l2_diff in value.items():
        if var not in invert_l2_diff_dict_ode:
            invert_l2_diff_dict_ode[var] = {}
        invert_l2_diff_dict_ode[var][key] = l2_diff


In [ ]:
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
linestyles = ["-", "--", ":", "-."]

t_Min = 150
for var in red_vars:
    # if var != "News":
    #     continue
    plt.figure()

    # Reset cycles for each variable
    color_cycle = cycle(colors)
    linestyle_cycle = cycle(linestyles)
    linestyle = next(linestyle_cycle)

    for idx, (run, value) in enumerate(invert_l2_diff_dict_ode[var].items()):
        # label = run[9:]
        label = run.split("/")[-1]
        # if run == "original_tol_010":
        #     label = "tol_0.1"
        # label = label.replace("_", "*")
        # if run == "original":
        #     label = "default_tol"

        common_time, value = value
        filtered_index = common_time > t_Min

        # Advance linestyle every time we exhaust the color list
        if idx % len(colors) == 0 and idx != 0:
            linestyle = next(linestyle_cycle)

        color = next(color_cycle)

        plt.plot(
            common_time[filtered_index],
            value[filtered_index],
            label=label,
            linestyle=linestyle,
            color=color,
        )

    plt.yscale("log")
    plt.xlabel("Time")
    plt.ylabel("L2 Diff Norm")
    plt.legend()
    # plt.title(f"base L=24 (tmin={t_Min}): {var}")
    plt.title(f"base maxt=0.01 (tmin={t_Min}): {var}")
    # plt.savefig(save_fig_folder_path / f"ode_tol_change_{var}_{t_Min}.png")
    # plt.savefig(Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/36_segs_R_conv/figures") / f"L_convg_{var}_{t_Min}_R={r}.png")
    plt.savefig(Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/max_step_size_test/figures") / f"max_step_size_base004_{var}_{t_Min}_R={r}.png")
    # plt.close()
plt.show()


### L and R points

#### L points

In [ ]:
base_run = base_path / "original/red_cce.h5"
base_run = base_path / "original_LMax_26/red_cce.h5"
l2_diff_dict_L = {
    "original_LMax_14": {},
    "original_LMax_16": {},
    "original_LMax_18": {},
    "original": {},
    "original_tol_Lmax21": {},
    "original_LMax_22": {},
    "original_LMax_24": {},
    # "original_LMax_26": {},
}

r = 100
base_run = base_path / f"different_ex_rad_R_convg/{r}_R_40_L_26/red_cce.h5"
l2_diff_dict_L = {
f"different_ex_rad_R_convg/{r}_R_40_L_16":{},
f"different_ex_rad_R_convg/{r}_R_40_L_18":{},
f"different_ex_rad_R_convg/{r}_R_40_L_20":{},
f"different_ex_rad_R_convg/{r}_R_40_L_22":{},
f"different_ex_rad_R_convg/{r}_R_40_L_24":{},
# f"different_ex_rad_R_convg/{r}_R_40_L_26":{},
}

red_vars = ["News", "Psi0", "Psi1", "Psi2", "Psi3", "Psi4", "Strain"]

with h5py.File(base_run, "r") as f:
    for key in l2_diff_dict_L.keys():
        run2 = base_path / f"{key}/red_cce.h5"
        with h5py.File(run2, "r") as f2:
            for var in red_vars:
                val1 = f[f"SpectreR0{r}.cce"][var][()]
                val2 = f2[f"SpectreR0{r}.cce"][var][()]

                t1, data1 = val1[:, 0], val1[:, 1:]
                t2, data2 = val2[:, 0], val2[:, 1:]

                # Define common time grid using intersection or union
                common_time = np.union1d(t1, t2)

                # Interpolate both data sets to the common time grid
                interp1 = interp1d(
                    t1,
                    data1,
                    axis=0,
                    bounds_error=False,
                    fill_value="extrapolate",
                    kind="cubic",
                )
                interp2 = interp1d(
                    t2,
                    data2,
                    axis=0,
                    bounds_error=False,
                    fill_value="extrapolate",
                    kind="cubic",
                )

                aligned1 = interp1(common_time)
                aligned2 = interp2(common_time)

                # Compute L2 norm of the difference across angular modes
                l2_diff = np.linalg.norm(aligned1 - aligned2, axis=1)
                l2_diff_dict_L[key][var] = (common_time, l2_diff)
                print(f"{key} {var}")

    t = f[f"SpectreR0{r}.cce"][red_vars[0]][:, 0]

invert_l2_diff_dict_L = {}
for key, value in l2_diff_dict_L.items():
    for var, l2_diff in value.items():
        if var not in invert_l2_diff_dict_L:
            invert_l2_diff_dict_L[var] = {}
        invert_l2_diff_dict_L[var][key] = l2_diff


In [ ]:
t_Min = 50
for var in red_vars:
    for run, value in invert_l2_diff_dict_L[var].items():
        common_time, value = value
        filtered_index = common_time > t_Min
        label = run.split("/")[-1]
        # if run == "original":
        #     label = "LMax_20(default)"
        # if run == "original_tol_Lmax21":
        #     label = "LMax_21"
        plt.plot(common_time[filtered_index], value[filtered_index], label=label)
    plt.yscale('log')
    plt.xlabel('Time')
    plt.ylabel('L2 Diff Norm')
    plt.legend()
    plt.title(f'base LMax_26 : {var}')
    plt.yscale('log')
    # plt.savefig(save_fig_folder_path / f"L_change_diff_{var}.png")
    # plt.savefig(Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/different_ex_rad_R_convg/figures") / f"r=100_L_covg_{var}.png")
    plt.clf()

#### R points

In [ ]:
base_run = base_path / "original/red_cce.h5"
base_run = base_path / "original_rad_pts_34/red_cce.h5"
l2_diff_dict_R = {
    "original_rad_pts_12": {},
    # "original_rad_pts_13": {},
    "original_rad_pts_14": {},
    # "original": {},
    "original_rad_pts_16": {},
    # "original_rad_pts_17": {},
    "original_rad_pts_18": {},
    "original_rad_pts_20": {},
    "original_rad_pts_22": {},
    "original_rad_pts_24": {},
    "original_rad_pts_26": {},
    "original_rad_pts_28": {},
    "original_rad_pts_30": {},
    # "original_rad_pts_34": {},
}

r = 100
base_run = base_path / "different_ex_rad_R_convg/100_50/red_cce.h5"
l2_diff_dict_R = {
"different_ex_rad_R_convg/100_24":{},
"different_ex_rad_R_convg/100_28":{},
"different_ex_rad_R_convg/100_30":{},
"different_ex_rad_R_convg/100_34":{},
"different_ex_rad_R_convg/100_38":{},
"different_ex_rad_R_convg/100_40":{},
"different_ex_rad_R_convg/100_42":{},
"different_ex_rad_R_convg/100_44":{},
"different_ex_rad_R_convg/100_46":{},
"different_ex_rad_R_convg/100_48":{},
# "different_ex_rad_R_convg/100_50":{},
}

red_vars = ["News", "Psi0", "Psi1", "Psi2", "Psi3", "Psi4", "Strain"]

with h5py.File(base_run, "r") as f:
    for key in l2_diff_dict_R.keys():
        run2 = base_path / f"{key}/red_cce.h5"
        with h5py.File(run2, "r") as f2:
            for var in red_vars:
                val1 = f[f"SpectreR0{r}.cce"][var][()]
                val2 = f2[f"SpectreR0{r}.cce"][var][()]

                t1, data1 = val1[:, 0], val1[:, 1:]
                t2, data2 = val2[:, 0], val2[:, 1:]

                # Define common time grid using intersection or union
                common_time = np.union1d(t1, t2)

                # Interpolate both data sets to the common time grid
                interp1 = interp1d(
                    t1,
                    data1,
                    axis=0,
                    bounds_error=False,
                    fill_value="extrapolate",
                    kind="cubic",
                )
                interp2 = interp1d(
                    t2,
                    data2,
                    axis=0,
                    bounds_error=False,
                    fill_value="extrapolate",
                    kind="cubic",
                )

                aligned1 = interp1(common_time)
                aligned2 = interp2(common_time)

                # Compute L2 norm of the difference across angular modes
                l2_diff = np.linalg.norm(aligned1 - aligned2, axis=1)
                l2_diff_dict_R[key][var] = (common_time, l2_diff)
                print(f"{key} {var}")

    t = f[f"SpectreR0{r}.cce"][red_vars[0]][:, 0]


invert_l2_diff_dict_R = {}
for key, value in l2_diff_dict_R.items():
    for var, l2_diff in value.items():
        if var not in invert_l2_diff_dict_R:
            invert_l2_diff_dict_R[var] = {}
        invert_l2_diff_dict_R[var][key] = l2_diff


In [ ]:
t_Min = 50
for var in red_vars:
    for run, value in invert_l2_diff_dict_R[var].items():
        common_time, value = value
        filtered_index = common_time > t_Min
        # label = run[9:]  # Remove 'original_' prefix for clarity
        label = run.split("/")[-1]  # Remove 'original_' prefix for clarity
        if run == "original":
            label = "rad_pts_15(default)"
        plt.plot(common_time[filtered_index], value[filtered_index], label=label)
    plt.yscale('log')
    plt.xlabel('Time')
    plt.ylabel('L2 Diff Norm')
    plt.legend()
    plt.title(f'base rad_pts_50 : {var}')
    plt.yscale('log')
    # plt.savefig(save_fig_folder_path / f"rad_pts_change_diff_{var}.png")
    # plt.savefig(Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/different_ex_rad_R_convg/figures") / f"r=100_R_convg_{var}.png")
    plt.clf()

In [ ]:
invert_l2_diff_dict_R['News']['original_rad_pts_30']

### Time step size

In [ ]:

dt_ode_tol = {
    "original_tol_1000": None,
    # "original_tol_500": None,
    "original_tol_100": None,
    # "original_tol_50": None,
    # "original_tol_20": None,
    "original_tol_10": None,
    # "original_tol_5": None,
    "original": None,
    # "original_tol_1.1": None,
    "original_tol_010": None,
    # "original_tol_0.2": None,
    # "original_tol_0.02": None,
    "original_tol_0.01": None,
    # "original_tol_0.001": None,
}
# dt_ode_tol = {
#     "rad_20_11factor_tol_1000": None,
#     "rad_20_11factor_tol_500": None,
#     "rad_20_11factor_tol_100": None,
#     "rad_20_11factor_tol_50": None,
#     "rad_20_11factor_tol_20": None,
#     "rad_20_11factor_tol_5": None,
#     "rad_20_11factor_tol_1.1": None,
#     "rad_20_11factor_tol_1": None,
#     "rad_20_11factor_tol_0.2": None,
#     # "rad_20_11factor_tol_0.02": None,
#     # "rad_20_11factor_tol_0.01": None,
#     # "rad_20_11factor_tol_0.001": None,
# }

red_vars = ["News", "Psi0", "Psi1", "Psi2", "Psi3", "Psi4", "Strain"]

for key in dt_ode_tol.keys():
    run2 = base_path / f"{key}/red_cce.h5"
    with h5py.File(run2, "r") as f2:
        t = f2['SpectreR0250.cce']['News'][1:, 0]
        dt = np.diff(f2['SpectreR0250.cce']['News'][:, 0])
        dt_ode_tol[key] = np.zeros((len(t), 2))
        dt_ode_tol[key][:, 0] = t
        dt_ode_tol[key][:, 1] = dt
        print(f"Done {key}")
        # dt_ode_tol[key] = f2['Cce']['CceTimeStep.dat'][()]




In [ ]:
start_index = 14000
for key, value in dt_ode_tol.items():
    plt.plot(value[start_index:, 0], value[start_index:, 1], label=key)
    # plt.plot(value[:, 0], np.abs(value[:, 1]-value[-1,1]), label=key)
plt.xlabel('Time')
plt.ylabel('Time Step Size')
# plt.yscale('log')
plt.legend()
# plt.savefig(save_fig_folder_path / "ode_tol_time_step_size.png")
# plt.title('Time Step Size for Different ODE Tolerances')

In [ ]:
# run2 = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/rad_20_11factor_tol_1000/red_cce.h5")
# run2 = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/rad_20_11factor_tol_500/red_cce.h5")
run2 = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/1rad30_ode_tol/r30_start_2000_tol_0.1/red_cce.h5")
run2 = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/max_step_size_test/max_step_0.01/red_cce.h5")
# run2 = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/original/red_cce.h5")
with h5py.File(run2, "r") as f2:
    print(f2.keys())
    # print(f2['SpectreR0250.cce'].keys())
    val = f2['SpectreR0250.cce']['News'][:,:2]
    # val = f2['Cce']['CceTimeStep.dat'][()]


In [ ]:
plt.plot(val[600:,0],np.diff(val[600-1:,0]))
# plt.yscale('log')

In [ ]:
WT_data = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/data/BondiCceR0250.h5")

with h5py.File(WT_data, "r") as f:
    print(f.keys())
    t = f['Beta.dat'][:,0]

In [ ]:
# save_path = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/noise_post_junk")/"del.h5"
# add_uniform_noise(
#     WT_data, save_path, noise_amp=1e-10, add_relative_noise=False, t_min=1000.0
# )

# with h5py.File(save_path, "r") as f:
#     with h5py.File(WT_data, "r") as f2:
#         for key in f.keys():
#             if "Version" in key:
#                 continue
#             start_index = np.searchsorted(t, 1000)
#             print(f"{key} : {np.linalg.norm(f[key][:start_index, 1:] - f2[key][:start_index, 1:])}")
#             # print(f"{key} : {np.linalg.norm(f[key][:, 1:])}")
#             # print(f"{key} : {np.linalg.norm(f2[key][:, 1:])}")
#             # print(f"{key} : {f[key].attrs['Legend']}")